# Notificações
## Importando Bibliotecas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, HTML
import collections
from random import random

## Dividindo o dataset por episódio

In [2]:
# separando por episódio e definido as semanas
def episode_split(df, id):
    dfcopy = df.copy()
    dfcopy = dfcopy.loc[dfcopy['episode_id'] == id]
    #Separa o episódio por semanas
    index_week = dfcopy.loc[dfcopy['goal'] == 19611, 'goal'].index.to_list()
    k = 0
    week = []
    for x in range(dfcopy.shape[0]):
        week.append(k)
        if x in index_week:
            k += 1
    w = pd.Series(week, name= "week")
    episode = pd.concat([dfcopy, w], axis = 1)
    return episode

## Estabelecendo a média diária ou semanal de cada equipe

In [3]:
def team_time(df):
    dfcopy = df.copy()
    weeks = []
    teams = dfcopy['team_id'].unique()
    for x in teams:
        weeks.append(len(dfcopy.loc[(dfcopy['team_id'] == x) & (dfcopy['goal'] == 19611), ['team_id']]))
    days = np.array(weeks) * 7
    return pd.DataFrame(pd.DataFrame({'team_id':teams, 'weeks':weeks, 'days':days}))
    

In [165]:
#Necessário estabelecer média de cada time pois o tempo corrente individualmente é diferente
def player_mean(df, interval='days'):
    dfcopy = df.copy()
    weeks = []
    time = team_time(dfcopy)
    aux = dfcopy[['team_id', 'player_id', 'score']]
    aux['score'] /= (aux['team_id'].map(time.set_index('team_id')[interval]))
    gp = aux.groupby(['player_id'], as_index=False).mean()

    player_pivot = gp.pivot(index= 'player_id', columns='team_id', values='score')
    player_pivot.columns = player_pivot.columns.astype('str')
    player_mean = player_pivot.sort_values(player_pivot.columns.to_list(), ascending=False, na_position='last')
    return player_mean


### Simulação de Monte Carlo com v.a. Normal para Score 

In [5]:

def score_simulation(df):
    dfcopy = df.copy()
    
    for j in dfcopy.columns.astype('str'):
        normal_va = np.random.normal(dfcopy[j].mean(), dfcopy[j].std(), len(dfcopy.index))
        normal_va = np.absolute(normal_va)
        normal_va = pd.DataFrame({'score':normal_va}).set_index(dfcopy.index)
    
        for i in dfcopy.index:
            for x in pd.isna(dfcopy.loc[dfcopy.index == i, [j]])[j].to_list():
                if x:
                    dfcopy.loc[dfcopy.index == i,[j]] = np.NaN
                else:
                    dfcopy.loc[dfcopy.index == i,[j]] = normal_va.loc[normal_va.index== i,:]['score']
    dfcopy = dfcopy.sort_values(dfcopy.columns.to_list(), ascending=False, na_position='last')
    return dfcopy

In [133]:
def get_key(dictionary ,value):
    for key, v in dictionary.items():
        if v == value:
            return key

In [7]:
results = pd.read_csv("../compiled_result.csv")

In [8]:
results.drop(labels=['goal_type', 'score'], axis=1, inplace=True)

## Selecionando o episódio com ID 35, pois contém maior volume de dados

In [9]:
episode = episode_split(results, 35)

## Estabelecendo um Score

In [10]:
#define pesos para os indicadores
idc_unq = episode['indicator_id'].unique()
np.array(idc_unq)
indicators = pd.DataFrame(data={ 'indicator_id': idc_unq, 'weight': [1, 2, 1, 1, 3, 2, 1]})

In [11]:
#Define o score dos players
episode = episode.merge(indicators , how='left')
episode['score'] = episode['result'] * episode['weight']

In [166]:
player_mean = player_mean(episode, interval='days')
player_mean = player_mean.drop(labels=['41'], axis=1) #Nescessário eliminar o time 41 pela falta de dados

/home/felipe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [13]:
player_sim = score_simulation(player_mean)

## Notificão pela média diária

In [171]:
data = pd.DataFrame()
dicio = {}
dicio_list = []
for j in player_mean.columns:
    k = 0
    for i in player_mean.index:
        value = player_mean.at[i, j]
        if not(pd.isna(value)):
            k += 1 #posição do player no seu time
            dicio.update({k:i})  #adiciona no dicionario posicao e id
    new_dict = dicio.copy()
    dicio_list.append(new_dict)

info_1 = []
notification_1 = []
notification_2 = []
notification_3 = []


m = 0
for j in player_mean.columns:
    for x in dicio_list:
        x_values = list(x.values())
        for k in x_values:
            player_m1 = player_mean.at[k,j]
            if not(pd.isna(player_m1)):
                m += 1
                for m in range(1 , len(x_values)+1):
                    player_m2 = player_mean.at[x[m], j]
                    if not(pd.isna(player_m2)):
                        mean_dif = np.absolute(player_m1 - player_m2)
                        current_score = player_sim.at[x[m], j]
                        if mean_dif != 0:
                            pass_score = mean_dif - current_score #Score que o player_m2 nescessita para passar o player_m1
                            indicador = indicators.copy()
                            indicador['score_to_pass'] = pass_score/indicador['weight']

                            info_1.append(f'Player1: {k} Player2: {x[m]} P1: {player_m1} para P2: {player_m2}')
                            notification_1.append(f'Player {x[m]} a pontuação média do Player {get_key(x, k)} é {int(mean_dif)} maior que a sua')
                            notification_2.append(f'Player {x[m]} faça {int(pass_score)} pontos para ultrapassar o Player {k} na posição {get_key(x, k)}')
                            #Pontuação e indicador referente para ultrapassar
                            for indi_id in indicador['indicator_id']:
                                indi_score = indicador.loc[indicador['indicator_id'] == indi_id, 'score_to_pass'].to_list()[0]
                                notification_3.append(f'Player {x[m]} faça {int(indi_score)} pontos referentes ao indicador {indi_id} para passar o player {k}')